In [1]:
import numpy as np
import pandas as pd
import math
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_med = pd.read_csv("data_wrangling_medical_2021_u6828533.csv", dtype={'postcode': str})
df_med.first_name.replace(to_replace=np.nan, value='nan*', inplace=True) # First name can be Nan. 
df_med.last_name.replace(to_replace=np.nan, value='null*', inplace=True) # Last name can be Null.

df_edu = pd.read_csv("data_wrangling_education_2021_u6828533.csv", dtype={'postcode': str})
df_edu.first_name.replace(to_replace=np.nan, value='nan*', inplace=True) # First name can be Nan.
df_edu.last_name.replace(to_replace=np.nan, value='null*', inplace=True) # Last name can be Null.

In [3]:
# sort by the number of NaNs of each row in descending order

df_med['null_count'] = df_med.isnull().sum(axis=1)
df_med=df_med.sort_values('null_count', ascending=True).drop('null_count', axis=1)

df_edu['null_count'] = df_edu.isnull().sum(axis=1)
df_edu=df_edu.sort_values('null_count', ascending=True).drop('null_count', axis=1)

In [4]:
# drop rows with duplicate ssn

df_med = df_med.drop_duplicates(subset=['ssn'], keep='first')
df_edu = df_edu.drop_duplicates(subset=['ssn'], keep='first')

df_merge = pd.merge(left=df_med, right=df_edu, how='inner', on='ssn')
unique_med = df_med[df_med.ssn.isin(df_edu.ssn) == False]
unique_edu = df_edu[df_edu.ssn.isin(df_med.ssn) == False]

print(f"common ssn in both datasets: {len(df_merge)}")
print(f"unique ssn in medical: {len(unique_med)}")
print(f"unique ssn in education: {len(unique_edu)}")

common ssn in both datasets: 16047
unique ssn in medical: 3953
unique ssn in education: 3194


In [5]:
def attribute_check(row, attribute):
    att_x = attribute + "_x"
    att_y = attribute + "_y"

    if pd.isnull(row[att_x]) and pd.isnull(row[att_y]):
        return True
    if row[att_x] == row[att_y]:
        return True
    else:
        return False

for check_attribute in ['first_name', 'middle_name', 'last_name', 'gender', 'birth_date', 'street_address', 'suburb', 'postcode', 'state', 'phone', 'email']:
    print(f"{check_attribute}: {len(df_merge.loc[df_merge.apply(lambda row: attribute_check(row, check_attribute), axis=1) == False])}")

first_name: 0
middle_name: 2831
last_name: 41
gender: 1638
birth_date: 0
street_address: 6875
suburb: 6781
postcode: 8590
state: 2964
phone: 8818
email: 6848


In [6]:
def attribute_update(row, attr):
    att_x = attr + "_x"
    att_y = attr + "_y"
    if row[att_x] == row[att_y]:
        return row[att_x]
    elif pd.isnull(row[att_x]):
        return row[att_y]
    elif pd.isnull(row[att_y]):
        return row[att_x]
    
    elif attr=='email':
        if "@" not in row[att_x]:
            return row[att_y]
        if "@" not in row[att_y]:
            return row[att_x]
        
    elif row['consultation_timestamp'] >= row['employment_timestamp']:
        return row[att_x]
    elif row['employment_timestamp'] >= row['consultation_timestamp']:
        return row[att_y]
    
for attribute in ['first_name', 'middle_name', 'last_name', 'gender', 'birth_date', 'street_address', 'suburb', 'postcode', 'state', 'phone', 'email']:
    df_merge[attribute] = df_merge.apply(lambda row: attribute_update(row,attribute), axis=1)

In [7]:
df_merge = df_merge.rename(columns={"rec_id_x": "rec_id_medical", "rec_id_y": "rec_id_education", "current_age": "age_at_employment"})

df_merge = df_merge[['ssn', 'rec_id_medical', 'rec_id_education', 
                     'first_name', 'middle_name', 'last_name',
                     'gender',  'birth_date', 'age_at_consultation', 
                     'age_at_employment','medicare_number', 'street_address', 
                     'suburb', 'postcode', 'state', 
                     'phone', 'email', 'marital_status', 
                     'height', 'weight', 'bmi', 
                     'blood_pressure', 'cholesterol_level', 'smoking_status', 
                     'clinical_notes', 'consultation_timestamp', 'education', 
                     'occupation', 'salary', 'credit_card_number', 
                     'years_of_experience', 'employment_timestamp']]

In [8]:
from itertools import combinations

# return list of all subsets of length r  
def rSubset(arr, r):
    return list(combinations(arr, r))
    
a1='middle_name'
a2='phone'
a3='email'
a4='marital_status'
a5='occupation'
a6='salary'
a7='credit_card_number'

arr = [a1, a2, a3, a4, a5, a6, a7]
r = 3
foo = []
for ax3 in rSubset(arr, r):    
    foo.append(df_merge.isnull().groupby([a for a in ax3]).size().to_frame())

In [9]:
"""
print results from foo[0] to foo[34] BY HAND
"""
foo[32]

0
marital_status occupation credit_card_number       
False          False      False               13979
True           False      False                 721
               True       True                 1347

In [10]:
df_merge.isnull().sum().sort_values(ascending=False).head(2)

salary            2313
marital_status    2068
dtype: int64

In [11]:
len(df_merge[df_merge.salary == -9999])

2375

In [12]:
# This line takes any salary values below 0 and replaces them with NaN
df_merge['salary'] = df_merge['salary'].mask(df_merge['salary'].between(-10000, 0))

# Find median of salary excluding NaN values
df_merge.salary.median(skipna=True)

85200.0

In [13]:
df_merge['salary'].fillna(85200, inplace=True)

In [14]:
df_merge['marital_status'].fillna("not-specified", inplace=True)

In [15]:
len(df_merge[df_merge.weight < 0])

1585

In [16]:
# Use bmi and height to calculate the correct weight

def weight(row):
    if row['weight'] < 0:
        row['weight'] = int(round(row['bmi'] * row['height'] * row['height'] / 100 / 100))
    return row['weight']

df_merge['weight'] = df_merge.apply(lambda row: weight(row), axis=1)

In [17]:
len(df_merge[~df_merge.state.isin(['nt', 'qld', 'nsw', 'sa', 'wa', 'vic', 'tas', 'act'])])

185

In [18]:
df_merge.state = df_merge.state.str.lower()

# strip the whitespace from suburb
df_merge['suburb'] = df_merge['suburb'].str.strip()
df_merge['suburb'] = df_merge['suburb'].str.replace('  ', ' ')
df_merge['suburb'] = df_merge['suburb'].str.lower()

In [19]:
df_merge['birth_year'] = pd.DatetimeIndex(df_merge['birth_date']).year 
df_merge['consultation_year'] = np.int64(df_merge['consultation_timestamp'].str[0:4])
len(df_merge[df_merge.age_at_consultation!=df_merge.consultation_year-df_merge.birth_year])

1632

In [20]:
df_merge.age_at_consultation = df_merge.consultation_year-df_merge.birth_year

In [21]:
len(df_merge[abs(df_merge.employment_timestamp.str.split('-', expand=True)[0].astype(int) - df_merge.consultation_timestamp.str.split('-', expand=True)[0].astype(int)) > 7])

965

In [22]:
df_merge = df_merge[abs(df_merge.employment_timestamp.str.split('-', expand=True)[0].astype(int) - df_merge.consultation_timestamp.str.split('-', expand=True)[0].astype(int)) <= 7]

In [23]:
len(df_merge[(df_merge.marital_status!='not-married') & (df_merge.age_at_consultation < 18)])

2535

In [24]:
df_merge.loc[(df_merge.age_at_consultation < 18), 'marital_status'] = 'not-married'

In [25]:
len(df_merge[(~pd.isnull(df_merge.credit_card_number)) & (df_merge.age_at_employment < 18)])

443

In [26]:
df_merge.loc[(df_merge.age_at_employment < 18), 'credit_card_number'] = 'not-available'

In [27]:
len(df_merge[(~pd.isnull(df_merge.occupation)) & (df_merge.age_at_employment < 18)])

443

In [28]:
df_merge.loc[(df_merge.age_at_employment < 18), 'occupation'] = 'not-available'

In [29]:
# Fill in missing phone and email details
df_merge['phone'].fillna("not-provided", inplace=True)
df_merge['email'].fillna("not-provided", inplace=True)
df_merge['middle_name'].fillna("not-specified", inplace=True)

df_merge.salary = df_merge.salary.astype('int64')

In [30]:
def convert_aqf(object):
    if object == 'diploma':
        return 5
    if object == 'advance-diploma':
        return 6
    if object == 'certificate-i':
        return 1
    if object == 'secondary-school':
        return 0
    if object == 'bachelor-degree':
        return 7
    if object == 'certificate-iii':
        return 3
    if object == 'graduate-certificate':
        return 8
    if object == 'certificate-ii':
        return 2
    if object == 'graduate-diploma':
        return 8
    if object == 'senior-secondary-school':
        return 0
    if object == 'certificate-iv':
        return 4
    if object == 'masters-degree':
        return 9
    if object == 'doctoral-degree':
        return 10
    if object == 'primary-school':
        return 0
    if object == 'pre-school':
        return 0

df_merge['education'] = df_merge['education'].apply(convert_aqf)

In [31]:
mms = MinMaxScaler()
df_merge['salary'] = mms.fit_transform(np.array(df_merge['salary']).reshape(-1,1))

In [32]:
df_merge.to_csv('data_wrangling_merged_2021_u6828533.csv', index=False)